<h2 align='center'>Codebasics ML Course: ML Flow Tutorial</h2>

In [21]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [22]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [23]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

#### Handle class imbalance

In [24]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train, return_counts=True)

(array([0, 1]), array([630,  70]))

### Track Experiments

In [25]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [26]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [27]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [28]:
# Initialize MLflow
mlflow.set_experiment("Anomally Detection")
mlflow.set_tracking_uri("http://127.0.0.1:5000")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run():        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2024/11/03 12:29:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/11/03 12:29:24 INFO mlflow.tracking._tracking_service.client: 🏃 View run trusting-kit-321 at: http://127.0.0.1:5000/#/experiments/923332001065858672/runs/40b30ef73d0049c8b1758e74c070ccc5.
2024/11/03 12:29:24 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/923332001065858672.
2024/11/03 12:29:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/11/03 12:29:29 INFO mlflow.tracking._tracking_service.client: 🏃 View run welcoming-pig-957 at: http://127.0.0.1:5000/#/experiments/923332001065858672/runs/6f93e2a595e642b486196436ed2b0661.
2024/11/03 12:29:29 INFO mlflow.tracking._tracking_service.client: 🧪 View ex

### Register the Model

In [35]:
model_name = 'XGB-smote'
run_id=input('Please type RunID')
model_uri = f'runs:/{run_id}/model_name'

with mlflow.start_run():
    mlflow.register_model(model_uri=model_uri, name=model_name)

Please type RunID 4a9c0aedb13c46ea833931f2c3b9f15e


Successfully registered model 'XGB-smote'.
2024/11/03 12:42:13 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-smote, version 1
Created version '1' of model 'XGB-smote'.
2024/11/03 12:42:13 INFO mlflow.tracking._tracking_service.client: 🏃 View run awesome-sow-622 at: http://127.0.0.1:5000/#/experiments/923332001065858672/runs/30bd2cb19c5e4e1dad6d674cb6a1a065.
2024/11/03 12:42:13 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/923332001065858672.


### Load the Model

In [36]:
model_name = "xgb-smote"
model_version = 1
model_uri = f"models:/{model_name}@challenger"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

RestException: INVALID_PARAMETER_VALUE: Registered model alias challenger not found.

### Transition the Model to Production

In [ ]:
current_model_uri = f"models:/{model_name}@challenger"
production_model_name = "anomale-detection-prod"

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=current_model_uri, dst_name=production_model_name)

In [ ]:
model_version = 1
prod_model_uri = f"models:/{production_model_name}@champion"

loaded_model = mlflow.xgboost.load_model(prod_model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

Please refer to following to learn more about model registry

https://mlflow.org/docs/latest/model-registry.html#model-registry-workflows to learn 